<a href="https://colab.research.google.com/github/jj132535/DeblurGAN-for-Video-Sharpness/blob/main/dncnn_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Get Dataset from Video File
import cv2
import os
import torch
import torch.nn as nn
import torch.utils.data as data
import numpy as np
import tqdm.notebook as tq
from torchvision import transforms
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

In [ ]:
# Paths to be configured by the user
video_path = "/content/16-9_CHUU_Strawberry_short.mp4"  # Path to the input video file
frame_output_path = "/content/frames"  # Path to save extracted frames
checkpoint_path = "/content/DNCNN_models/30.tar"  # Path to the trained model checkpoint
result_save_path = "/content/denoised_frames"  # Path to save denoised frames
output_video_path = "/content/denoised_video.mp4"  # Path to save the final denoised video

In [ ]:
# 수정된 코드
import torch.utils.data as torch_data

In [ ]:




os.makedirs(frame_output_path, exist_ok=True)

# Extract frames from video
cap = cv2.VideoCapture(video_path)
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_filename = os.path.join(frame_output_path, f"frame_{frame_count:04d}.png")
    cv2.imwrite(frame_filename, frame)
    frame_count += 1

cap.release()

# Noise Transform
class NoiseTransform(object):
  def __init__(self, size=180, mode="training"):
    super(NoiseTransform, self).__init__()
    self.size = size
    self.mode = mode

  def gaussian_noise(self, img):
    mean = 0
    stddev = 25
    noise = torch.zeros(img.size()).normal_(mean, stddev/255.)
    return noise

  def __call__(self, img):
    if self.mode in ["training", "validation"]:
      self.gt_transform = transforms.Compose([
        transforms.Resize((self.size, self.size), interpolation=2),
        transforms.ToTensor()])
      self.noise_transform = transforms.Compose([
        transforms.Resize((self.size, self.size), interpolation=2),
        transforms.ToTensor(),
        transforms.Lambda(self.gaussian_noise),
      ])
      return self.gt_transform(img), self.noise_transform(img)

    elif self.mode == "testing":
      self.gt_transform = transforms.Compose([
        transforms.Resize((self.size, self.size), interpolation=2),
        transforms.ToTensor()])
      return self.gt_transform(img)
    else:
      return NotImplementedError

class NoiseDataset(torch_data.Dataset):  # 여기에서 torch_data로 변경
    def __init__(self, root_path, size):
        super(NoiseDataset, self).__init__()
        self.root_path = root_path
        self.size = size
        self.transforms = None
        self.examples = [os.path.join(root_path, f) for f in os.listdir(root_path) if f.endswith(".png")]

    def set_mode(self, mode):
        self.mode = mode
        self.transforms = NoiseTransform(self.size, mode)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        file_name = self.examples[idx]
        image = Image.open(file_name)

        if self.mode == "testing":
            input_img = self.transforms(image)
            sample = {"img": input_img, "file_name": "image_%06d.png" % int(os.path.basename(file_name).split('_')[1].split('.')[0])}
        else:
            clean, noise = self.transforms(image)
            sample = {"img": clean, "noise": noise}

        return sample

# Simplified DNCNN network
class DNCNN(nn.Module):
  def __init__(self, in_planes=3, blocks=17, hidden=64, kernel_size=3, padding=1, bias=False):
    super(DNCNN, self).__init__()
    self.conv_f = nn.Conv2d(in_channels=in_planes, out_channels=hidden, kernel_size=kernel_size, padding=padding, bias=bias)
    self.conv_h = nn.Conv2d(in_channels=hidden, out_channels=hidden, kernel_size=kernel_size, padding=padding, bias=bias)
    self.conv_l = nn.Conv2d(in_channels=hidden, out_channels=in_planes, kernel_size=kernel_size, padding=padding, bias=bias)

    self.bn = nn.BatchNorm2d(hidden)
    self.relu = nn.ReLU(inplace=True)

    self.hidden_layer = self.mk_hidden_layer(blocks)

  def mk_hidden_layer(self, blocks=17):
    layers = []
    for _ in range(blocks-2):
      layers.append(self.conv_h)
      layers.append(self.bn)
      layers.append(self.relu)
    return nn.Sequential(*layers)

  def forward(self, x):
    out = self.conv_f(x)
    out = self.relu(out)
    out = self.hidden_layer(out)
    out = self.conv_l(out)
    return out



In [ ]:
# Testing Phase

os.makedirs(result_save_path, exist_ok=True)

use_cuda = torch.cuda.is_available()

test_dataset = NoiseDataset(frame_output_path, 128)
test_dataset.set_mode("testing")

test_dataloader = torch_data.DataLoader(test_dataset, batch_size=1, shuffle=False)

net = DNCNN()

if use_cuda:
  net.to('cuda')
else:
  print('CUDA not available, using CPU instead')

if not os.path.exists(checkpoint_path):
    # Train the model if checkpoint does not exist
    print('Checkpoint not found. Training the model...')
    train_dataset = NoiseDataset(frame_output_path, 128)
    train_dataset.set_mode("training")
    train_dataloader = torch_data.DataLoader(train_dataset, batch_size=4, shuffle=True)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, eps=1e-08)

    if use_cuda:
        net.to('cuda')
        criterion.to('cuda')

    for epoch in range(5):  # Example epoch count
        net.train()
        for data in train_dataloader:
            if use_cuda:
                clean, noise = data["img"].to('cuda'), data["noise"].to('cuda')
            else:
                clean, noise = data["img"], data["noise"]

            optimizer.zero_grad()
            noisy_img = clean + noise
            pred_noise = net(noisy_img)
            loss = criterion(pred_noise, noise)
            loss.backward()
            optimizer.step()


CUDA not available, using CPU instead


In [ ]:
# Save the trained model
torch.save(net.state_dict(), checkpoint_path)
print(f"Model saved at {checkpoint_path}")

net.load_state_dict(torch.load(checkpoint_path, map_location='cuda' if use_cuda else 'cpu'))
model = nn.DataParallel(net)

model.eval()

# Process frames and save denoised frames
for i, data in enumerate(tq.tqdm(test_dataloader)):
    if use_cuda:
        img = data["img"].to('cuda')
    else:
        img = data["img"]

    file_name = data["file_name"]

    # file_name이 리스트일 경우 첫 번째 요소만 사용
    if isinstance(file_name, list):
        file_name = file_name[0]

    with torch.no_grad():
        out_test = torch.clamp(img - model(img), 0., 1.)

    out_img = transforms.ToPILImage()(out_test.squeeze().cpu())
    out_img.save(os.path.join(result_save_path, file_name))


# Combine denoised frames into video
frame_files = sorted([f for f in os.listdir(result_save_path) if f.endswith(".png")])
first_frame = cv2.imread(os.path.join(result_save_path, frame_files[0]))
height, width, layers = first_frame.shape

out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 60, (width, height))

for frame_file in frame_files:
    frame = cv2.imread(os.path.join(result_save_path, frame_file))
    out.write(frame)

out.release()


Model saved at /content/DNCNN_models/30.tar


<ipython-input-28-473e18ce9d02>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(checkpoint_path, map_location='cuda' if use_cuda else 'cpu'))

  0%|          | 0/600 [00:00<?, ?it/s]